In [2]:
import numpy as np
import matplotlib.pyplot as plt

## Game Logic

In [ ]:
board = np.full((3, 3), '' ,dtype=str)
player = "Y"

def make_move(board, move):
    global player
    x,y = move[0], move[1]
    if board[x][y] == '':
        board[x][y] = player
    print(board)

def check_line(line, player):
    # Returns True if all elements in the line match the player aka X or O
    return np.all(line == player)

def check_draw(board):
    if np.all(board != ''):
        return True
    return False
def check_win(board):
    global player
    #Check rows
    for row in range(3):
        if check_line(board[row, :], player):
            return True
    #Check Columns
    for col in range(3):
        if check_line(board[:, col], player):
            return True
    #Check diagonal
    if check_line(np.diag(board), player):
        return True
    #Check other diagonal
    if check_line(np.diag(np.flip(board)), player):
        return True
    #No win
    return False

def draw_board(board, opponent:str):
    plt.figure(figsize=(5, 5))
    plt.xlim(0, 3)
    plt.ylabel(0, 3)

    plt.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)
    
    # Vertical
    plt.plot([1,1], [3,3], color='black', linewidth=2)
    plt.plot([2,2], [0,3], color='black', linewidth=2)
    # Horizontal
    plt.plot([0,3], [1,1], color='black', linewidth=2)
    plt.plot([0,3], [2,2], color='black', linewidth=2)

    #Place things
    for row in range(3):
        for col in range(3):
            symbol = board[row, col]

            if symbol != '':
                x_pos = col + 0.5
                y_pos = 2.5 - row
                #Color
                color = 'red' if symbol == 'X' else 'blue'
                #draw
                plt.text(x_pos, y_pos, symbol, fontsize=40, ha='center', va='center', color=color)
    plt.title("Tic Tac Toe vs " + opponent)
    plt.show()


make_move(board, (0,0))
make_move(board, (0,1))
make_move(board, (0,2))
print(check_win(board))
print(check_draw(board))
draw_board(board, "MEEEEEEEEEEE")
# while True:
#     input()

[['Y' '' '']
 ['' '' '']
 ['' '' '']]
[['Y' 'Y' '']
 ['' '' '']
 ['' '' '']]
[['Y' 'Y' 'Y']
 ['' '' '']
 ['' '' '']]
True
False


## Game Player